# Lab 1: Planar transformations and image rectification

The two main goals of this first lab are the following:

1) Get more familiar with the hierarchy of 2D transformations. <br>

2) Image rectification: Removal of the projective distortion of an image of a planar object.

This notebook combines some text cells (Markdown cells) and code cells. Some parts of the code need to be completed. All tasks you need to complete are marked in <span style='color:LightGreen'> LightGreen.  </span>

## **1. Image transformations**

In this first part of the lab you will apply different types of 2D transformations to a given image. For that, you first need to create a function that applies a homography to an image.

<span style='color:LightGreen'> - Create the function  *apply_H* that gets as input a homography and
an image and returns the image transformed by the homography. </span>

Note: The size of the transformed image has to be automatically set so that it contains the whole transformed image.
You will need to interpolate the image values at some points, for that,
you may use the function *scipy.ndimage.map_coordinates*

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage import map_coordinates
from numpy import linalg as LA
from utils import line_draw, plot_img

# Translations are not visible !!!

def apply_H(Image, Homography):
    
    h, w = Image.shape[:2]

    # Corners in homogeneous coordinates
    corners = np.array([[     0, w - 1,     0,  w - 1],
                        [     0,     0, h - 1,  h - 1],
                        [     1,     1,     1,      1]
                        ])
    
    # Transform corners to find output image bounds
    transformed_corners = Homography @ corners
    transformed_corners /= transformed_corners[2] 

    # Compute the output image's size
    min_x, min_y = np.floor(transformed_corners[:2].min(axis=1)).astype(int)
    max_x, max_y = np.ceil(transformed_corners[:2].max(axis=1)).astype(int)
    out_w, out_h = max_x - min_x,  max_y - min_y

    # Create a grid of coordinates in the output image space
    x, y = np.meshgrid(np.arange(out_w) + min_x, np.arange(out_h) + min_y)
    coords = np.stack([x.ravel(), y.ravel(), np.ones_like(x.ravel())])
    
    # Map coordinates back to input image using *inverse homography*
    inv_H = LA.inv(Homography)
    src_coords = inv_H @ coords
    src_coords /= src_coords[2] 

    # Interpolate pixel values
    map_x, map_y = src_coords[0].reshape(out_h, out_w), src_coords[1].reshape(out_h, out_w)
    transformed = [
        map_coordinates(Image[..., c], [map_y, map_x], order=1, mode='constant', cval=0)
        for c in range(3)
    ]
    return np.stack(transformed, axis=-1)

### **1.1 Similarities**

<span style='color:LightGreen'> - Complete the code below by generating a matrix H that produces a similarity transformation. </span>

In [ ]:
from math import pi

theta = pi/6 # angle
s = .5 # scale
tx = 1000; ty = 0 # translation (NOT VISIBLE)

H_Similarity = np.array([
            [s * np.cos(theta),     -s * np.sin(theta),      tx],
            [s * np.sin(theta),      s * np.cos(theta),      ty],
            [                0,                      0,       1]
        ])

img_path = "./Data/0005_s.png"
I = Image.open(img_path)
I_sim = apply_H(np.array(I), H_Similarity)
plot_img(I)
plot_img(I_sim)

### **1.2 Affinities**

<span style='color:LightGreen'> - Complete the code below by generating a matrix H that produces an affine transformation.  </span>

<span style='color:LightGreen'> - Decompose the affinity in four transformations: two
rotations, a scale, and a translation (you may use function *numpy.linalg.svd* for that).  </span>

<span style='color:LightGreen'> - Verify that the product of the four previous transformations
produces the same matrix H as above.  </span>

In [ ]:
# Parameters
theta, phi = np.pi / 9, np.pi / 12      # Rotation angles
sx, sy = 0.75, 0.5                       # Scaling factors
tx, ty = 300, 200                       # Translation values (NOT VISIBLE)

# Rotation matrices
R1 = np.array([[np.cos(theta), -np.sin(theta)], 
               [np.sin(theta),  np.cos(theta)]])  

R2 = np.array([[np.cos(phi), -np.sin(phi)], 
               [np.sin(phi),  np.cos(phi)]])     

# Scaling matrix
S = np.diag([sx, sy]) 

# Combine transformations
A = R2 @ S @ R1  

# Affine matrix with translation
H_Affinity = np.eye(3)
H_Affinity[:2, :2] = A
H_Affinity[:2, 2] = [tx, ty]

I_aff = apply_H(np.array(I), H_Affinity)

plot_img(I)
plot_img(I_aff)

<span style='color:LightGreen'> - Verify that the proper sequence of the four previous
transformations, applied over the image `I` **one by one**, produces the same transformed image as before.  </span>

In [ ]:
# Rotation matrix 1
R1 = np.array([[np.cos(theta), -np.sin(theta), 0], 
               [np.sin(theta),  np.cos(theta), 0],
               [            0,              0, 1]])
I_aff_R1 = apply_H(np.array(I), R1)

# Scale matrix
S = np.array([[sx,  0,  0], 
              [ 0, sy,  0], 
              [ 0,  0,  1]])
I_aff_R1_S = apply_H(I_aff_R1, S)

# Rotation matrix 2
R2 = np.array([[np.cos(phi), -np.sin(phi), 0], 
               [np.sin(phi),  np.cos(phi), 0],
               [          0,            0, 1]])
I_aff_R1_S_R2 = apply_H(I_aff_R1_S, R2)

# Translation matrix
T = np.array([[1, 0, tx], 
              [0, 1, ty], 
              [0, 0,  1]]).astype(float)
I_aff = apply_H(I_aff_R1_S_R2, T)

# Plot the results at each step
plot_img(I)
plot_img(I_aff_R1)
plot_img(I_aff_R1_S)
plot_img(I_aff_R1_S_R2)
plot_img(I_aff)

As we can see there's a small discrepancy as the absolute coordinates are used when we've already scaled the image. We can achieve the exact same results by removing the empty rows and cols.

In [ ]:
I_aff = I_aff[I_aff.any(axis=(1,2)), :, :]
I_aff = I_aff[:, I_aff.any(axis=(0,2)), :]

plot_img(I_aff)

### **1.3 Projective transformations (Homographies)**

<span style='color:LightGreen'> - Complete the code below by generating a matrix H that produces a projective transformation.  </span>

In [ ]:
# Parameters for projective transformation
h11, h12, h13 = 1.5, 0.2, -0.5    # can the parameters be explained in a simple way?
h21, h22, h23 = 0.3, 1.1, 0.2   
h31, h32, h33 = 0.001, 0.002, 1  

# Define the projective transformation matrix
H_projective = np.array([
    [h11, h12, h13],
    [h21, h22, h23],
    [h31, h32, h33]
])

I_proj = apply_H(np.array(I), H_projective)

plot_img(I)
plot_img(I_proj)

## **2. Affine Rectification**

This step is needed in order to rectify an image in a stratified way, where we first perform affine rectification (current section) and then metric rectification (Section 3).

### **2.1 Vanishing points estimated semi-automatically**

First, we will perform affine rectification by computing the vanishing points in a semi-automatic way. Line segments in the image will be detected automatically and then pairs of segments corresponding to imaged parallel lines will be manually selected. The segments are detected by the Line Segment Detector algorithm (paper [1], demo and code available in [2]). The result of this algorithm in the images of interest is already provided. The initial and final points of each detected segment are provided in a text file. The code below shows how to read the corresponding points for a certain segment given its index.  

[1] R. Grompone von Gioi, J. Jakubowicz, J.-M. Morel, G. Randall. LSD: a Line Segment Detector. , Image Processing On Line, 2, , pp. 35–55, 2012.

[2] http://www.ipol.im/pub/art/2012/gjmr-lsd/

In [ ]:
# load images and lines
img_path = "./Data/0000_s.png"
I = Image.open(img_path)
img2_path = "./Data/0000_s_lines.jpg"
Il = Image.open(img2_path)
#plot_img(Il)

lines_path = "./Data/0000_s_info_lines.txt"
A = np.loadtxt(lines_path)

# points of interest
i = 423 # line index (starting from 0)
p1 = [A[i, 0], A[i, 1], 1] # initial point in line i
p2 = [A[i, 2], A[i, 3], 1] # final point in line i
i = 239
p3 = [A[i, 0], A[i, 1], 1]
p4 = [A[i, 2], A[i, 3], 1]
i = 711
p5 = [A[i, 0], A[i, 1], 1]
p6 = [A[i, 2], A[i, 3], 1]
i = 564
p7 = [A[i, 0], A[i, 1], 1]
p8 = [A[i, 2], A[i, 3], 1]

points = [p1, p2, p3, p4, p5, p6, p7, p8]
points = [np.array(p) for p in points]

<span style='color:LightGreen'> - Compute the lines l1, l2, l3, l4, that pass through the different pairs of points.  </span>

In [ ]:
l1 = np.cross(points[0], points[1])  # Line between points[0] and points[1]
l2 = np.cross(points[2], points[3])  # Line between points[2] and points[3]
l3 = np.cross(points[4], points[5])  # Line between points[4] and points[5]
l4 = np.cross(points[6], points[7])  # Line between points[6] and points[7]

In [ ]:
# show the chosen lines in the image
canv = ImageDraw.Draw(I)
line_draw(l1, canv, I.size)
line_draw(l2, canv, I.size)
line_draw(l3, canv, I.size)
line_draw(l4, canv, I.size)

# The displayed lines will alter image I so we have to reopen the original image after the plot
plot_img(I)
I = Image.open(img_path)

<span style='color:LightGreen'> - Compute the homography that affinely rectifies the image. </span>

In [ ]:
# Compute intersection points using cross product of lines
p_infinity1 = np.cross(l1, l2)  # Intersection of l1 and l2
p_infinity2 = np.cross(l3, l4)  # Intersection of l3 and l4

# Normalize the intersection points to make them homogeneous
p_infinity1 /= p_infinity1[2]
p_infinity2 /= p_infinity2[2]

# Construct the homography
l_inf = np.cross(p_infinity1, p_infinity2)
H = np.eye(3)
H[2] = l_inf / l_inf[2] 

I_aff_01 = apply_H(np.array(I), H)
plot_img(I)
plot_img(I_aff_01)

<span style='color:LightGreen'> - Compute the transformed lines lr1, lr2, lr3, lr4 and
      show the transformed lines in the transformed image. </span>

In [ ]:
def transform_line(line, H):
    """Transform a line using the homography matrix."""
    H_inv = np.linalg.inv(H)  # Inverse of the homography
    line_transformed = np.dot(H_inv.T, line)  # Transform the line
    return line_transformed / np.linalg.norm(line_transformed[:2])

lr1 = transform_line(l1, H)
lr2 = transform_line(l2, H)
lr3 = transform_line(l3, H)
lr4 = transform_line(l4, H)

I_aff = Image.fromarray(I_aff_01)
canv = ImageDraw.Draw(I_aff)
line_draw(lr1, canv, I_aff.size)
line_draw(lr2, canv, I_aff.size)
line_draw(lr3, canv, I_aff.size)
line_draw(lr4, canv, I_aff.size)

# The displayed lines will alter image I so we have to reopen the original image after the plot
plot_img(I_aff)

<span style='color:LightGreen'> - To evaluate the results, compute the angle between the different pair 
      of lines before and after the image transformation. </span>

In [ ]:
import math
def angle_between_lines(line1, line2):
    """Compute the angle between two lines."""
    # Extract coefficients
    a1, b1, _ = line1
    a2, b2, _ = line2

    # Compute the dot product and magnitudes
    dot_product = a1 * a2 + b1 * b2
    magnitude1 = np.sqrt(a1**2 + b1**2)
    magnitude2 = np.sqrt(a2**2 + b2**2)

    # Compute the cosine of the angle
    cos_theta = dot_product / (magnitude1 * magnitude2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)  # Clip to avoid numerical issues

    # Compute the angle in radians and then convert to degrees
    theta = math.acos(cos_theta)
    return math.degrees(theta)

# Original lines
angles_before = {
    "l1_l2": angle_between_lines(l1, l2),
    "l3_l4": angle_between_lines(l3, l4),
}

# Transformed lines
angles_after = {
    "lr1_lr2": angle_between_lines(lr1, lr2),
    "lr3_lr4": angle_between_lines(lr3, lr4),
}

# Display results
print("Angles before transformation:", angles_before)
print("Angles after transformation:", angles_after)

<span style='color:LightGreen'> - Verify, experimentally, that the cross-ratio is preserved after the image rectification (you may choose the endpoints of some detected line segments). </span>

### **2.2 Vanishing points estimated automatically** 

The vanishing points in an image can be estimated automatically. In this lab, we will use the orthogonal vanishing points estimated by the technique proposed in [3] (code available in [4]), which actually uses the line segments estimated with the LSD algorithm [1] used before.

In this section, we will work with the image 'friends.jpeg'. The estimated vanishing points obtained by [3] are provided in the file 'friends_vps.out' (see code below for more details).

[3] Xiaohu Lu, Jian Yao, Haoang Li, Yahui Liu. 2-Line Exhaustive Searching for Real-Time Vanishing Point Estimation in Manhattan World. IEEE/CVF Winter Conference on Applications of Computer Vision (WACV), 2017.

[4] https://github.com/rayryeng/XiaohuLuVPDetection

<span style='color:LightGreen'> - Perform affine rectification of this image using the appropriate vanishing points automatically detected. **Justify your choice** of vanishing points. </span>

In [ ]:
# Visualize image 'friends.jpeg'
img_path = "./Data/friends.jpeg"
I = Image.open(img_path)
plot_img(I)
# Visualize image 'friends_vps.jpeg' which shows the different line segments detected by LSD [1] and groups
# them in three different colors according to the different vanishing points they have been associated.
img2_path = "./Data/friends_vps.jpeg"
Ivp = Image.open(img2_path)
plot_img(Ivp)

In [ ]:
# Load the three orthogonal vanishing points from image 'friends.jpeg' (estimated by [3,4])
vps = np.loadtxt('./Data/friends_vps.out', delimiter=',')
print(vps)
# 'vps' is a numpy array where each row is a vanishing point and each column is a coordinate.
# The first, second and third rows in 'vps' correspond to the vanishing points corresponding to,
# respectively, red, LightGreen, and blue directions.

In [ ]:
p_blue= [vps[2, 0], vps[2, 1], 1]
p_red = [vps[0, 0], vps[0, 1], 1]

l_inf = np.cross(p_blue, p_red)

# Construct the homography
H = np.eye(3)
H[2] = l_inf / l_inf[2] 

I_aff = apply_H(np.array(I), H)
plot_img(I)
plot_img(I_aff)

## **3. Metric Rectification**

### **3.1 Metric rectification after the affine rectification (stratified solution)**

We will work with image 0000.

<span style='color:LightGreen'> - Write the code that performs the metric rectification (after the affine rectification). </span>

As qualitative evaluation method you can display the images (before and after the metric rectification) with the chosen lines printed on it.
      
<span style='color:LightGreen'> - Compute the angles between the pair of lines before and after rectification. Comment the result. </span>



In [ ]:
def compute_symmetric_matrix(orthogonal_pairs):
    """Compute the symmetric matrix S from two pairs of orthogonal lines."""
    A = []
    b = []

    for l, m in orthogonal_pairs:
        # Equation: (l1*m1, l1*m2 + l2*m1) * [s11, s12] = -l2*m2
        A.append([l[0] * m[0], l[0] * m[1] + l[1] * m[0]])
        b.append(-l[1] * m[1])

    A = np.array(A)
    b = np.array(b)

    # Solve for [s11, s12]
    s = np.linalg.lstsq(A, b, rcond=None)[0]
    s11, s12 = s
    S = np.array([[s11, s12], [s12, 1]])  # Construct the symmetric matrix S
    return S

def compute_affine_transform(S):
    """Compute the affine transform matrix H2 from S."""
    # Eigen-decomposition: S = U D U^T
    eigvals, eigvecs = np.linalg.eigh(S)
    
    D = np.diag(np.sqrt(eigvals)) # Square root of eigenvalues
    A = eigvecs @ D @ eigvecs.T    # A = U sqrt(D) U^T
    return A

In [ ]:
lr1 /= np.linalg.norm(lr1[2])
lr2 /= np.linalg.norm(lr2[2])
lr3 /= np.linalg.norm(lr3[2])
lr4 /= np.linalg.norm(lr4[2])

orthogonal_pairs = [
    (np.array([lr1[0], lr1[1], lr1[2]]), np.array([lr2[0], lr2[1], lr2[2]])),  # Pair 1
    (np.array([lr3[0], lr3[1], lr3[2]]), np.array([lr4[0], lr4[1], lr4[2]]))   # Pair 2
]

# Step 1: Compute S
S = compute_symmetric_matrix(orthogonal_pairs)

# Step 2: Compute A
A = compute_affine_transform(S)

# Step 3: Form H2
H2 = np.eye(3)
H2[:2, :2] = A
H2 = np.linalg.inv(H2)

I_metric = apply_H(np.array(I_aff_01), H2)
lm1 = transform_line(lr1, H2)
lm2 = transform_line(lr2, H2)
lm3 = transform_line(lr3, H2)
lm4 = transform_line(lr4, H2)

I_metric= Image.fromarray(I_metric)
canv = ImageDraw.Draw(I_metric)
point_color = (0, 0, 255)
line_draw(lm1, canv, I_metric.size)
line_draw(lm2, canv, I_metric.size)
line_draw(lm3, canv, I_metric.size)
line_draw(lm4, canv, I_metric.size)

plot_img(I_metric)

In [ ]:
angles_before = {
    "l1_l2": angle_between_lines(l1, l2),
    "l3_l4": angle_between_lines(l3, l3),
}

# Transformed lines
angles_after = {
    "lm1_lm3": angle_between_lines(lm1, lm2),
    "lm3_lm4": angle_between_lines(lm3, lm4),
}

# Display results
print("Angles before transformation:", angles_before)
print("Angles after transformation:", angles_after)

## **4. Affine and Metric Rectification of the left facade of image 0001**

<span style='color:LightGreen'> - Write the code that rectifies the left facade of image 0001 with
      the stratified method.  </span>
      
Note: For a better visualization of the result crop the initial image so that only the left facade is visible.

<span style='color:LightGreen'> - Show the (properly) transformed lines that are used in every step.  </span>


In [ ]:
img_path = "./Data/0001_s.png"
I = Image.open(img_path)
I = I.crop((0, 0, 500, I.height))
img2_path = "./Data/0001_s_lines.jpg"
Il = Image.open(img2_path)
plot_img(I)

lines_path = "./Data/0001_s_info_lines.txt"
A = np.loadtxt(lines_path)

# points of interest
i = 613 # line index (starting from 0)
p1 = [A[i, 0], A[i, 1], 1] # initial point in line i
p2 = [A[i, 2], A[i, 3], 1] # final point in line i
i = 158
p3 = [A[i, 0], A[i, 1], 1]
p4 = [A[i, 2], A[i, 3], 1]
i = 540
p5 = [A[i, 0], A[i, 1], 1]
p6 = [A[i, 2], A[i, 3], 1]
i = 644
p7 = [A[i, 0], A[i, 1], 1]
p8 = [A[i, 2], A[i, 3], 1]


points = [p1, p2, p3, p4, p5, p6, p7, p8]
points = [np.array(p) for p in points]


In [ ]:
def find_intersection(line1, line2):
    # Line 1: a1*x + b1*y + c1 = 0
    # Line 2: a2*x + b2*y + c2 = 0
    a1, b1, c1 = line1
    a2, b2, c2 = line2
    
    # Create the coefficient matrix and the constants vector
    A = np.array([[a1, b1], [a2, b2]])
    B = np.array([-c1, -c2])
    
    # Solve the system of linear equations
    intersection = np.linalg.solve(A, B)
    intersection = [intersection[0], intersection[1], 1]
    
    return intersection


# Find intersection
intersection_point1 = find_intersection(l1, l3)
intersection_point2 = find_intersection(l2, l4)
intersection_point3 = find_intersection(l1, l4)
intersection_point4 = find_intersection(l2, l3)

l1 = np.cross(points[0], points[1])  # Line between points[0] and points[1]
l2 = np.cross(points[2], points[3])  # Line between points[2] and points[3]
l3 = np.cross(points[4], points[5])  # Line between points[4] and points[5]
l4 = np.cross(points[6], points[7])  # Line between points[6] and points[7]
l5 = np.cross(intersection_point1, intersection_point2)  # Line between points[6] and points[7]
l6 = np.cross(intersection_point3, intersection_point4)  # Line between points[6] and points[7]




# show the chosen lines in the image
canv = ImageDraw.Draw(I)
point_color = (0, 0, 255)
line_draw(l1, canv, I.size)
line_draw(l2, canv, I.size)
line_draw(l3, canv, I.size)
line_draw(l4, canv, I.size)
line_draw(l5, canv, I.size)
line_draw(l6, canv, I.size)


# The displayed lines will alter image I so we have to reopen the original image after the plot
plot_img(I)
I = Image.open(img_path)

print(type(l1))
I = Image.open(img_path)
I = I.crop((0, 0, 500, I.height))


In [ ]:
# Compute intersection points using cross product of lines
p_infinity1 = np.cross(l1, l2)  # Intersection of l1 and l2
p_infinity2 = np.cross(l3, l4)  # Intersection of l3 and l4


# Normalize the intersection points to make them homogeneous
p_infinity1 /= p_infinity1[2]
p_infinity2 /= p_infinity2[2]


l_inf = np.cross(p_infinity1, p_infinity2)

# Construct the homography
H = np.eye(3)
H[2] = l_inf / l_inf[2] 

I_aff_01 = apply_H(np.array(I), H)
plot_img(I_aff_01)

In [ ]:


lr1 /= np.linalg.norm(lr1[2])
lr3 /= np.linalg.norm(lr3[2])
lr5 /= np.linalg.norm(lr5[2])
lr6 /= np.linalg.norm(lr6[2])




orthogonal_pairs = [
    (np.array([lr1[0], lr1[1], lr1[2]]), np.array([lr3[0], lr3[1], lr3[2]])),  # Pair 1
    (np.array([lr5[0], lr5[1], lr5[2]]), np.array([lr6[0], lr6[1], lr6[2]]))   # Pair 2
]

S = compute_symmetric_matrix(orthogonal_pairs)

# Step 2: Compute A
A = compute_affine_transform(S)

# Step 3: Form H2
H2 = np.eye(3)
H2[:2, :2] = A
H2 = np.linalg.inv(H2)


I_metric = apply_H(np.array(I_aff_01), H2)


s_x = 1239 / 967
s_y = 1736 / 1188



lm1 = transform_line(lr1, H2)
lm2 = transform_line(lr2, H2)
lm3 = transform_line(lr3, H2)
lm4 = transform_line(lr4, H2)
lm5 = transform_line(lr5, H2)
lm6 = transform_line(lr6, H2)

'''
lm1 = [lm1[0]*s_x, lm1[1]*s_y, lm1[2]* np.sqrt(s_x**2 + s_y**2)]
lm2 = [lm2[0]*s_x, lm2[1]*s_y, lm1[2]* np.sqrt(s_x**2 + s_y**2)]
lm3 = [lm3[0]*s_x, lm3[1]*s_y, lm1[2]* np.sqrt(s_x**2 + s_y**2)]
lm4 = [lm4[0]*s_x, lm4[1]*s_y, lm1[2]* np.sqrt(s_x**2 + s_y**2)]
lm5 = [lm5[0]*s_x, lm5[1]*s_y, lm5[2]* np.sqrt(s_x**2 + s_y**2)]
lm6 = [lm6[0]*s_x, lm6[1]*s_y, lm6[2]* np.sqrt(s_x**2 + s_y**2)]
'''
    

I_metric= Image.fromarray(I_metric)
canv = ImageDraw.Draw(I_metric)
point_color = (0, 0, 255)
line_draw(lm1, canv, I_metric.size)
line_draw(lm2, canv, I_metric.size)
line_draw(lm3, canv, I_metric.size)
line_draw(lm4, canv, I_metric.size)
line_draw(lm5, canv, I_metric.size)
line_draw(lm6, canv, I_metric.size)

print(lm1)

print(I_metric.size)
plot_img(I_metric)

## **5. OPTIONAL: Metric Rectification in a single step**

<span style='color:LightGreen'> - Write the code that performs metric rectification of the white facade of image 0000 in a single step (algorithm pages 55-57, Hartley-Zisserman book). </span>

Note: Use 5 pairs of orthogonal lines. You may consider that windows are square.